In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [43]:
data = pd.read_csv("./data/title.basics.tsv/data.tsv", sep="\t", header=0)

In [44]:
data.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [54]:
data[data.genres.str.contains("Comedy")==True].head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
13,tt0000014,short,The Sprinkler Sprinkled,L'arroseur arrosé,0,1895,\N,1,"Comedy,Short"
18,tt0000019,short,The Clown Barber,The Clown Barber,0,1898,\N,\N,"Comedy,Short"
31,tt0000033,short,Trick Riding,La voltige,0,1895,\N,1,"Comedy,Documentary,Short"


In [55]:
data2 = pd.read_csv("./data/title.ratings.tsv/data.tsv", sep="\t", header=0)

In [56]:
data2.head()

,tconst,averageRating,numVotes
0,tt0000001,5.8,1491
1,tt0000002,6.3,181
2,tt0000003,6.6,1129
3,tt0000004,6.4,110
4,tt0000005,6.2,1830


In [60]:
data2[data.genres.str.contains("Comedy")==True].head()

C:\Users\pomaz\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,tconst,averageRating,numVotes
2,tt0000003,6.6,1129
4,tt0000005,6.2,1830
13,tt0000014,7.2,3926
18,tt0000019,6.6,13
31,tt0000033,5.6,756


In [58]:
data3 = pd.read_csv("./data/title.akas.tsv/data.tsv", sep="\t", header=0)

C:\Users\pomaz\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
data3.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
1,tt0000001,2,Καρμενσίτα,GR,\N,\N,\N,0
2,tt0000001,3,Карменсита,RU,\N,\N,\N,0
3,tt0000001,4,Carmencita,US,\N,\N,\N,0
4,tt0000001,5,Carmencita,\N,\N,original,\N,1


In [75]:
data3[data3.region == "US"].head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
3,tt0000001,4,Carmencita,US,\N,\N,\N,0
10,tt0000002,6,The Clown and His Dogs,US,\N,\N,literal English title,0
23,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
26,tt0000005,4,Blacksmith Scene #1,US,\N,alternative,\N,0
27,tt0000005,5,Blacksmithing,US,\N,\N,informal alternative title,0


In [76]:
"US" in list(data3.region.unique().tolist())

True

In [79]:
len(data3[data3.region == "US"].title.unique().tolist())

707373